In [ ]:
"""This script will convert N*4 hdf5 files into 4 hdf5 files that each represent N files of one polarization."""
# Python Standard Library Packages
import os
# import copy
import glob

# Community Developed Packages
import numpy as np
# import astropy.units as u
# import astropy.constants as c
# import matplotlib.pyplot as plt

# HERA Collaboration Packages
# import hera_pspec as hp
# from hera_pspec.data import DATA_PATH
from pyuvdata import UVData
# from hera_cal import redcal

In [ ]:
hdf5_DATA_PATH = '/lustre/aoc/projects/hera/afortino/zen_2458111_one_1pol_HH_uvOCRS_hdf5/'
SAVE_PATH = '/lustre/aoc/projects/hera/afortino/zen_2458111_all_1pol_HH_uvOCRS_hdf5'
EXTENSION = 'uvOCRS'

In [ ]:
hdf5_DFILES = {
    'xx': sorted(glob.glob(os.path.join(hdf5_DATA_PATH, 'zen.2458111.?????.xx.HH.{ext}.hdf5'.format(
        ext=EXTENSION)))),
    'xy': sorted(glob.glob(os.path.join(hdf5_DATA_PATH, 'zen.2458111.?????.xy.HH.{ext}.hdf5'.format(
        ext=EXTENSION)))),
    'yx': sorted(glob.glob(os.path.join(hdf5_DATA_PATH, 'zen.2458111.?????.yx.HH.{ext}.hdf5'.format(
        ext=EXTENSION)))),
    'yy': sorted(glob.glob(os.path.join(hdf5_DATA_PATH, 'zen.2458111.?????.yy.HH.{ext}.hdf5'.format(
        ext=EXTENSION))))}

In [ ]:
for pol in sorted(hdf5_DFILES):
    uvd = UVData()
    uvdf = UVData()
    uvd.read_uvh5(hdf5_DFILES[pol])
    uvdf.read_uvh5(hdf5_DFILES[pol][-1])
    
    JDt0 = np.round(np.unique(uvd.time_array)[0], 5)
    JDtf = np.round(np.unique(uvdf.time_array)[0], 5)
    JD = str(int(JDt0))
    JDt0 = '{:.5f}'.format(JDt0).split('.')[-1]
    JDtf = '{:.5f}'.format(JDtf).split('.')[-1]
    numfiles = str(len(hdf5_DFILES[pol]))
    
    uvd.extra_keywords['JDt0'] = JDt0
    uvd.extra_keywords['JDtf'] = JDtf
    uvd.extra_keywords['JD'] = JD
    uvd.extra_keywords['numfiles'] = numfiles
    uvd.extra_keywords['ext'] = EXTENSION
    
    hdf5 = 'zen.{JD}.{JDt0}_{numfiles}_{JDtf}.{pol}.HH.{ext}'.format(
        JD=JD,
        JDt0=JDt0,
        numfiles=numfiles,
        JDtf=JDtf,
        pol=pol,
        ext=EXTENSION)
    hdf5 = os.path.join(SAVE_PATH, hdf5)
    print hdf5
    uvd.write_uvh5(hdf5)